In [1]:
!pip install --upgrade --quiet pip
!pip install --quiet xgboost==1.7.1
!pip install --quiet boto3
!pip install --quiet sagemaker

### Setup

In [2]:
%%time

import os
import boto3
import re
import json
import sagemaker
from sagemaker import get_execution_role
import pandas as pd

region = boto3.Session().region_name

role = get_execution_role()

bucket = sagemaker.Session().default_bucket()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
CPU times: user 1.61 s, sys: 254 ms, total: 1.86 s
Wall time: 2.65 s


In [3]:
prefix = 'sagemaker/xgboost-byo'
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket)

### Training

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [37]:
data = pd.read_csv("WeatherData.csv", index_col=0)
data

,mintemp_c,maxtemp_c,precip_mm,sunshine,gust_kph,daily_will_it_rain,tom_will_it_rain,wind_kph,humidity,pressure_mb,...,temp_c,PersonID,sex,age,height,weight,BMI,freqOfExercise,CI,Target
0,17.9,35.2,0.0,12.3,48,0,0,6,20,1006.3,...,26.6,1,0,40,175.80,87.75,28.39,6,2.307144,-1.805114
1,17.9,35.2,0.0,12.3,48,0,0,6,20,1006.3,...,26.6,2,0,74,170.76,84.59,29.01,2,3.343987,-1.806799
2,17.9,35.2,0.0,12.3,48,0,0,6,20,1006.3,...,26.6,3,0,24,176.49,81.89,26.29,4,2.459838,-1.802157
3,17.9,35.2,0.0,12.3,48,0,0,6,20,1006.3,...,26.6,4,0,39,175.88,88.58,28.64,0,4.243744,-1.807501
4,17.9,35.2,0.0,12.3,48,0,0,6,20,1006.3,...,26.6,5,0,61,173.37,89.91,29.91,1,4.302597,-1.810043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47575,22.9,33.4,17.8,6.2,93,1,1,15,67,1008.0,...,30.3,6,1,37,162.47,74.20,28.11,5,2.384639,-0.963873
47576,22.9,33.4,17.8,6.2,93,1,1,15,67,1008.0,...,30.3,7,1,30,162.40,70.60,26.77,2,3.139030,-0.964535
47577,22.9,33.4,17.8,6.2,93,1,1,15,67,1008.0,...,30.3,8,1,34,162.24,69.47,26.39,1,2.461945,-0.962891
47578,22.9,33.4,17.8,6.2,93,1,1,15,67,1008.0,...,30.3,9,1,22,163.19,66.04,24.80,0,4.014209,-0.963780


In [56]:
#separate the response variable and predict variable
X = data.drop("Target", axis=1)
y = data["Target"]

#split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
import xgboost as xgb
import sklearn as sk

xgb_model = xgb.XGBRegressor(
    max_depth=9, learning_rate=0.15, n_estimators=400)

xgb_model.fit(X_train, Y_train)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/data.py:299: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/data.py:301: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categorical

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.15, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=9, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=400, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [38]:
#predict the result
XGBr_pred = xgb_model.predict(X_test)

#MSE
XGBr_mse = mean_squared_error(Y_test, XGBr_pred)
print(f'Mean Squared Error of XGBoost Regression: {XGBr_mse}')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/data.py:299: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/data.py:301: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categorical

Mean Squared Error of XGBoost Regression: 4.601607941144223e-06


### Saving the model

In [39]:
model_file_name = "locally-trained-xgboost-model"
xgb_model.save_model(model_file_name)

In [40]:
!tar czvf model.tar.gz $model_file_name

locally-trained-xgboost-model


### Upload model to S3

In [41]:
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

### Set up hosting 

In [42]:
#####Container used is very different
#containers = {
#              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest'
#              }
#container = containers[boto3.Session().region_name]

#Refer: https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html#sagemaker.image_uris.retrieve

from sagemaker.image_uris import retrieve
container=retrieve(framework ="xgboost",region= boto3.Session().region_name, version="1.7-1")
container

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1'

In [69]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_url = 'https://s3-{}.amazonaws.com/{}/{}'.format(region,bucket,key)

sm_client = boto3.client('sagemaker')

primary_container = {
    'Image': container,
    'ModelDataUrl': model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response2['ModelArn'])

arn:aws:sagemaker:us-east-1:952556617726:model/locally-trained-xgboost-model2023-11-25-10-31-37
CPU times: user 30.7 ms, sys: 0 ns, total: 30.7 ms
Wall time: 599 ms


In [68]:
from time import gmtime, strftime

endpoint_config_name = 'XGBoostEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'InitialVariantWeight':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

XGBoostEndpoint-2023-11-25-10-31-37
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:952556617726:endpoint-config/xgboostendpoint-2023-11-25-10-31-37


In [46]:
%%time
import time

endpoint_name = 'XGBoostEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

XGBoostEndpoint-2023-11-25-10-31-37
arn:aws:sagemaker:us-east-1:952556617726:endpoint/xgboostendpoint-2023-11-25-10-31-37
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:952556617726:endpoint/xgboostendpoint-2023-11-25-10-31-37
Status: InService
CPU times: user 123 ms, sys: 26.5 ms, total: 150 ms
Wall time: 4min 1s


### Testing

In [16]:
runtime_client = boto3.client('sagemaker-runtime', region_name=region)

In [19]:
# pred = xgb_model.predict(X_test)
import numpy as np

In [64]:
point_X = X_test.iloc[0]
point_X = np.expand_dims(point_X, axis=0)

np.savetxt("test_point.csv", point_X, delimiter=",")

label = Y_test.iloc[0]

In [65]:
import json
from itertools import islice
import struct

file_name = "test_point.csv"

with open(file_name, "r") as f:
    payload = f.read().strip()
    
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload
)

result = response["Body"].read().decode("ascii")

### Closing endpoint
Have to be done when finished to avoid stray instance from running

In [45]:
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'baa9f999-c581-402f-896f-b3b5239e6d2f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'baa9f999-c581-402f-896f-b3b5239e6d2f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 25 Nov 2023 10:31:18 GMT'},
  'RetryAttempts': 0}}